<a href="https://colab.research.google.com/github/chandirakumar/Text_Summarization/blob/main/Auto_Generated_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install datasets contractions -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 7.5 MB/s eta 0:00:00


In [2]:
# prompt: create full script for abstractive text summarization which will tokenize the data and finetune the bart cnn model and calculate rogue score

import transformers
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from datasets import load_dataset, load_from_disk, load_metric, load_metric
from tqdm import tqdm
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from contractions import contractions_dict
import re
from matplotlib import pyplot as plt
from transformers import BertTokenizer, BertForMaskedLM, BertModel, BartForConditionalGeneration, BartTokenizer
from nltk.corpus import brown
import string
from random import randint
from wordcloud import WordCloud
import seaborn as sns
from PIL import Image
from google.colab import drive
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, AutoConfig, AutoTokenizer
from torch.utils.data import DataLoader


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the CNN/Daily Mail dataset
cnn_dm_dataset = load_dataset("cnn_dailymail",  '3.0.0', split="train")

In [ ]:

# Preprocess the data
def preprocess_data(dataset):
    # Convert the text to lowercase
    dataset["article"] = str(dataset["article"]).lower()
    dataset["highlights"] = str(dataset["highlights"]).lower()

    # Remove punctuation
    dataset["article"] = str(dataset["article"]).replace(r"[^a-zA-Z\s]", "")
    dataset["highlights"] = str(dataset["highlights"]).replace(r"[^a-zA-Z\s]", "")

    # Remove stop words
    stop_words = set(stopwords.words("english"))
    dataset["article"] = dataset.map(lambda x: " ".join([word for word in str(x['article']).split() if word not in stop_words]))
    dataset["highlights"] = dataset.map(lambda x: " ".join([word for word in str(x['highlights']).split() if word not in stop_words]))

    # Lemmatize the words
    lemmatizer = nltk.WordNetLemmatizer()
    dataset["article"] = dataset.map(lambda x: " ".join([lemmatizer.lemmatize(word) for word in str(x["article"]).split()]))
    dataset["highlights"] = dataset.map(lambda x: " ".join([lemmatizer.lemmatize(word) for word in str(x["highlights"]).split()]))

    # Truncate the articles and highlights to a maximum length
    dataset["article"] = dataset.map(lambda x: x["article"][:1024])
    dataset["highlights"] = dataset.map(lambda x: x["highlights"][:128])

    return dataset

cnn_dm_dataset = preprocess_data(cnn_dm_dataset)

In [ ]:

# Create a tokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

# Create a model
model_bart_cnn = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base").to(device)

# Create a data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model_bart_cnn)

# Create the dataloaders
train_dataloader = DataLoader(cnn_dm_dataset["train"], batch_size=16, shuffle=True, collate_fn=data_collator)
test_dataloader = DataLoader(cnn_dm_dataset["test"], batch_size=16, shuffle=False, collate_fn=data_collator)

# Set the optimizer
optimizer = torch.optim.AdamW(model_bart_cnn.parameters(), lr=1e-5)

# Set the learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=3)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    # Train
    model_bart_cnn.train()
    train_loss = 0
    for batch in tqdm(train_dataloader):
        batch = batch.to(device)
        outputs = model_bart_cnn(**batch)
        loss = outputs.loss
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation
    model_bart_cnn.eval()
    val_loss = 0
    for batch in tqdm(test_dataloader):
        batch = batch.to(device)
        outputs = model_bart_cnn(**batch)
        loss = outputs.loss
        val_loss += loss.item()

    # Logging
    print(f"Epoch {epoch+1}: Train loss = {train_loss/len(train_dataloader):.4f}, Val loss = {val_loss/len(test_dataloader):.4f}")

    # Update the learning rate
    scheduler.step(val_loss)

# Save the model
model_bart_cnn.save_pretrained("my_bart_cnn_model")


In [ ]:

# Load the model
model_bart_cnn = AutoModelForSeq2SeqLM.from_pretrained("my_bart_cnn").to(device)


In [ ]:
# Generate summaries
def generate_summary(article_text):
    inputs = tokenizer.encode("summarize: " + article_text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model_bart_cnn.generate(inputs, max_length=120, min_length=30, length_penalty=2.0, num_beams=6, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [ ]:
# Evaluate the model
def evaluate_model(model, dataset):
    rouge = load_metric("rouge")
    for article, summary in zip(dataset["article"], dataset["highlights"]):
        generated_summary = generate_summary(article)
        scores = rouge.compute(predictions=[generated_summary], references=[summary])
        print(f"Original summary: {summary}")
        print(f"Generated summary: {generated_summary}")
        print(f"ROUGE-L: {scores['rouge-l']['l'].item():.4f}")
        print(f"ROUGE-2: {scores['rouge-2']['f'].item():.4f}")
        print(f"ROUGE-1: {scores['rouge-1']['f'].item():.4f}")

evaluate_model(model_bart_cnn, cnn_dm_dataset["test"])

In [ ]:
# prompt: abstractive text summarization with t5 base train and finetune the cnn data for test summarization


import transformers
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from tqdm import tqdm
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from contractions import contractions_dict
import re
from matplotlib import pyplot as plt
from transformers import BertTokenizer, BertForMaskedLM, BertModel, BartForConditionalGeneration, BartTokenizer
from nltk.corpus import brown
import string
from random import randint
from wordcloud import WordCloud
import seaborn as sns
from PIL import Image
from google.colab import drive
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, AutoConfig, AutoTokenizer
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the CNN/Daily Mail dataset
cnn_dm_dataset = load_dataset("cnn_dailymail",  '3.0.0', split="train")

# Preprocess the data
def preprocess_data(dataset):
    # Convert the text to lowercase
    dataset["article"] = dataset["article"].lower()
    dataset["highlights"] = dataset["highlights"].lower()

    # Remove punctuation
    dataset["article"] = str(dataset["article"]).replace(r"[^a-zA-Z\s]", "")
    dataset["highlights"] = str(dataset["highlights"]).replace(r"[^a-zA-Z\s]", "")

    # Remove stop words
    stop_words = set(stopwords.words("english"))
    dataset["article"] = dataset.map(lambda x: " ".join([word for word in str(x['article']).split() if word not in stop_words]))
    dataset["highlights"] = dataset.map(lambda x: " ".join([word for word in str(x['highlights']).split() if word not in stop_words]))

    # Lemmatize the words
    lemmatizer = nltk.WordNetLemmatizer()
    dataset["article"] = dataset.map(lambda x: " ".join([lemmatizer.lemmatize(word) for word in str(x["article"]).split()]))
    dataset["highlights"] = dataset.map(lambda x: " ".join([lemmatizer.lemmatize(word) for word in str(x["highlights"]).split()]))

    # Truncate the articles and highlights to a maximum length
    dataset["article"] = dataset.map(lambda x: x["article"][:1024])
    dataset["highlights"] = dataset.map(lambda x: x["highlights"][:128])

    return dataset

cnn_dm_dataset = preprocess_data(cnn_dm_dataset)

# Create a tokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

# Create a model
model_bart_cnn = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base").to(device)

# Create a data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model_bart_cnn)

# Create the dataloaders
train_dataloader = DataLoader(cnn_dm_dataset["train"], batch_size=16, shuffle=True, collate_fn=data_collator)
test_dataloader = DataLoader(cnn_dm_dataset["test"], batch_size=16, shuffle=False, collate_fn=data_collator)

# Set the optimizer
optimizer = torch.optim.AdamW(model_bart_cnn.parameters(), lr=1e-5)

# Set the learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=3)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    # Train
    model_bart_cnn.train()
    train_loss = 0
    for batch in tqdm(train_dataloader):
        batch = batch.to(device)
        outputs = model_bart_cnn(**batch)
        loss = outputs.loss
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation
    model_bart_cnn.eval()
    val_loss = 0
    for batch in tqdm(test_dataloader):
        batch = batch.to(device)
        outputs = model_bart_cnn(**batch)
        loss = outputs.loss
        val_loss += loss.item()

    # Logging
    print(f"Epoch {epoch+1}: Train loss = {train_loss/len(train_dataloader):.4f}, Val loss = {val_loss/len(test_dataloader):.4f}")

    # Update the learning rate
    scheduler.step(val_loss)

# Save the model
model_bart_cnn.save_pretrained("my_bart_cnn_model")

# Load the model
model_bart_cnn = AutoModelForSeq2SeqLM.from_pretrained("my_bart_cnn").to(device)

# Generate summaries
def generate_summary(article_text):
    inputs = tokenizer.encode("summarize: " + article_text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model_bart_cnn.generate(inputs, max_length=120, min_length=30, length_penalty=2.0, num_beams=6, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Evaluate the model
def evaluate_model(model, dataset):
    rouge = load_metric("rouge")
    for article, summary in zip(dataset["article"], dataset["highlights"]):
        generated_summary = generate_summary(article)
        scores = rouge.compute(predictions=[generated_summary], references=[summary])
        print(f"Original summary: {summary}")
        print(f"Generated summary: {generated_summary}")
        print(f"ROUGE-L: {scores['rouge-l']['l'].item():.4f}")
        print(f"ROUGE-2: {scores['rouge-2']['f'].item():.4f}")
        print(f"ROUGE-1: {scores['rouge-1']['f'].item():.4f}")

evaluate_model(model_bart_cnn, cnn_dm_dataset["test"])
